# Tensorflow基本使用

## [一个最基本的例子](http://wiki.jikexueyuan.com/project/tensorflow-zh/tutorials/mnist_pros.html)

In [ ]:
# -*- coding:utf-8 -*-
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x,W) + b)

y_ = tf.placeholder("float", [None,10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
init = tf.global_variables_initializer()

**上述为定义的过程**


In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(64)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print('accuracy:{}'.format(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})))


In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

# 偏好值初始化
def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

# 卷积过程
def conv2d(x,W):
    return tf.nn.conv2d(input=x,filter=W,strides=[1,1,1,1],padding='SAME')

# 池化过程
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],
                         strides=[1,2,2,1],padding='SAME')

def relu(x):
    return tf.nn.relu(x)

def softmax(x):
    return tf.nn.softmax(x)

In [ ]:
cnn_x = tf.placeholder(tf.float32,[None,784],name='cnn_x')
cnn_y = tf.placeholder(tf.float32, [None,10],name='cnn_y')

image = tf.reshape(cnn_x,shape=[-1,28,28,1])

w_1 = weight_variable([5,5,1,32])
conv1 = conv2d(image,w_1)
pool = max_pool_2x2(conv1)
relu1 = relu(pool)

w_2 = weight_variable([5,5,32,64])

conv2 = conv2d(relu1,w_2)

w_fc1 = weight_variable([14*14*64,1024])
b_fc1 = bias_variable([1024])
conv2_flat = tf.reshape(conv2, [-1, 14 * 14 * 64])
h_fc1 = tf.nn.relu(tf.matmul(conv2_flat, w_fc1) + b_fc1)


#全连接层fc2
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
# 直接经过softmax进行分类
prediction = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)


cross_entropy = tf.reduce_mean(-tf.reduce_sum(cnn_y * tf.log(prediction),
                                              reduction_indices=[1]))  # loss
train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy)
init = tf.global_variables_initializer()



In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(10000):
        batch_xs, batch_ys = mnist.train.next_batch(64)
        test=sess.run(image,feed_dict={cnn_x:batch_xs,cnn_y:batch_ys})
        correct_prediction = tf.equal(tf.argmax(cnn_y,1), tf.argmax(prediction,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        if i % 50 == 0:
            print(i)
            print('accuracy:{}'.format(sess.run(accuracy,feed_dict={cnn_x:mnist.test.images, cnn_y:mnist.test.labels})))
   
    

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(64)
        sess.run(train_step2, feed_dict={cnn_x: batch_xs, cnn_y: batch_ys})
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(cnn_y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print('accuracy:{}'.format(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})))

In [ ]:
cnn_x = tf.placeholder(tf.float32,[None,784],name='cnn_x')/255
cnn_y = tf.placeholder(tf.float32, [None,10],name='cnn_y')
x_image = tf.reshape(cnn_x, [-1, 28, 28, 1])
keep_prob = tf.placeholder(tf.float32)
## conv1 layer ##
W_conv1 = weight_variable([5,5, 1,32]) # patch 5x5, in size 1, out size 32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # output size 28x28x32
h_pool1 = max_pool_2x2(h_conv1)                                         # output size 14x14x32

# ## conv2 layer ##
W_conv2 = weight_variable([5,5, 32, 64]) # patch 5x5, in size 32, out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # output size 14x14x64
h_pool2 = max_pool_2x2(h_conv2)                                         # output size 7x7x64

# ## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
# [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# ## fc2 layer ##
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2)+ b_fc2)
# prediction = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2
                           

In [ ]:



with tf.Session() as sess:
    sess.run(init)
    
    for i in range(10000):
        batch_xs, batch_ys = mnist.train.next_batch(64)
        print(batch_xs.shape)
        test=sess.run(image,feed_dict={cnn_x:batch_xs,cnn_y:batch_ys,keep_prob:0.5})
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(cnn_y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        if i % 50 == 0:
            print(i)
            print('accuracy:{}'.format(sess.run(accuracy,feed_dict={cnn_x:mnist.test.images, cnn_y:mnist.test.labels})))

In [ ]:
#第一个卷积池为3x3的卷积池，输出为7个
W_1 = weight_variable([3,3,1,3])
#经过卷积过程
conv1 = conv2d(image,W_1)
#池化过程
pool1 = max_pool_2x2(conv1)
#relu过程
relu1 = relu(pool1)

#第二个卷积池，3x3，输出为一个图
W_2 = weight_variable([3,3,3,1])
conv2 = conv2d(relu1,W_2)

#全连接层fc1
w_fc1 = weight_variable([14*14,1024])
b_fc1 = bias_variable([1024])
conv2_flat = tf.reshape(conv2, [-1, 14 * 14 * 1])
h_fc1 = tf.nn.relu(tf.matmul(conv2_flat, w_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, 1)

#全连接层fc2
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
# 直接经过softmax进行分类
y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


cross_entropy = tf.reduce_mean(-tf.reduce_sum(cnn_y * tf.log(prediction),
                                              reduction_indices=[1]))  # loss
train_step2 = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

init = tf.global_variables_initializer()